In [ ]:
import sys

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/gdrive') # this is apparently sensitive to double or single quotes
    PROJECT_PATH = "/content/gdrive/My Drive/Colab Notebooks/DL4H Team 30 Project/"
else:
    PROJECT_PATH = ""

REQUIREMENTS_FILE = f"{PROJECT_PATH}requirements.txt"

Mounted at /content/gdrive


In [ ]:
pip install -r "{REQUIREMENTS_FILE}"

In [ ]:
import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText
# import glove
# from glove import Corpus
from mittens import GloVe
from statistics import mean
import random

import collections
import gc

import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
#from keras.layers import Flatten, Dense, Dropout, Input, concatenate, merge, Activation, Concatenate, LSTM, GRU
from keras.layers import Flatten, Dense, Dropout, Input, concatenate, Activation, Concatenate, LSTM, GRU   # merge replaced by concatenate
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, BatchNormalization, GRU, Convolution1D, LSTM
from keras.layers import UpSampling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPool1D

# from keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam

from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau
#from keras.utils import np_utils
from keras.utils import to_categorical
#from keras.backend.tensorflow_backend import set_session, clear_session, get_session
from tensorflow.keras.backend import clear_session
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.regularizers import L2
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization
import tensorflow as tf


from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# set seed
SEED = 10

def set_seed(seed):
    """Set seed"""
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(SEED)


# Define different sub folder paths
DATA_PATH = f"{PROJECT_PATH}data"
EMBEDDING_PATH = f"{PROJECT_PATH}embeddings"
RESULTS_PATH = f"{PROJECT_PATH}results"

In [ ]:
%%time

x_train_ts = pd.read_pickle(f"{DATA_PATH}/full_x_train.pkl")
x_dev_ts = pd.read_pickle(f"{DATA_PATH}/full_x_dev.pkl")
x_test_ts = pd.read_pickle(f"{DATA_PATH}/full_x_test.pkl")

y_train = pd.read_pickle(f"{DATA_PATH}/full_y_train.pkl")
y_dev = pd.read_pickle(f"{DATA_PATH}/full_y_dev.pkl")
y_test = pd.read_pickle(f"{DATA_PATH}/full_y_test.pkl")

word2vec_embeddings = pd.read_pickle(f"{DATA_PATH}/full_word2vec_embeddings.pkl")
fasttext_embeddings = pd.read_pickle(f"{DATA_PATH}/full_fasttext_embeddings.pkl")
concatenated_embeddings = pd.read_pickle(f"{DATA_PATH}/full_concat_embeddings.pkl")

train_ids = pd.read_pickle(f"{DATA_PATH}/full_train_ids.pkl")
dev_ids = pd.read_pickle(f"{DATA_PATH}/full_dev_ids.pkl")
test_ids = pd.read_pickle(f"{DATA_PATH}/full_test_ids.pkl")

CPU times: user 17.4 s, sys: 8.54 s, total: 25.9 s
Wall time: 1min 13s


In [ ]:
# Initialize results dataframe
results_df = pd.DataFrame(columns=['Task', 'Model', 'Embedding', 'Iteration', 'AUC', 'AUPRC', 'ACC', 'F1'])

## Proposed model

In [ ]:
def make_predictions(model, test_data):
    probs = model.predict(test_data)
    y_pred = [1 if i>=0.5 else 0 for i in probs]
    return probs, y_pred

def save_scores(predictions, probs, ground_truth, embed_name, problem_type,
                iteration, hidden_unit_size, type_of_ner, model_name):

    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)

    result_dict = {'Task': problem_type,
                   'Model': model_name,
                   'Embedding' : embed_name,
                   'Iteration': iteration,
                   'AUC': auc,
                   'AUPRC': auprc,
                   'ACC': acc,
                   'F1': F1}

    # set as global to update results df
    global results_df
    results_df.loc[len(results_df)] = result_dict
    #results_df.drop_duplicates(subset=['Task', 'Model', 'Embedding', 'Iteration'],keep='last',inplace = True) # may not be necessary
    results_df.to_csv(f"{PROJECT_PATH}/results/results.csv", index=False)

    print(auc, auprc, acc, F1)

def print_scores(predictions, probs, ground_truth, model_name, problem_type, iteration, hidden_unit_size):
    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)

    result_dict = {}
    result_dict['auc'] = auc
    result_dict['auprc'] = auprc
    result_dict['acc'] = acc
    result_dict['F1'] = F1

    print ("AUC: ", auc, "AUPRC: ", auprc, "F1: ", F1)

def get_subvector_data(size, embed_name, data):
    """
    Turn embeddings into matrix of the dimensions vector_size x size
    """
    # specify vector size
    if embed_name == "concat":
        vector_size = 200
    else:
        vector_size = 100

    x_data = {}
    for k, v in data.items():
        # get the number of additional vectors needed for padding
        number_of_additional_vector = len(v) - size
        vector = []
        for i in v:
            vector.append(i)
        # If vector length is less than required size, pad with zero vectors
        # else, slice the vector to keep only the required size
        if number_of_additional_vector < 0:
            number_of_additional_vector = np.abs(number_of_additional_vector)

            temp = vector[:size]
            for i in range(0, number_of_additional_vector):
                temp.append(np.zeros(vector_size))
            x_data[k] = np.asarray(temp)
        else:
            x_data[k] = np.asarray(vector[:size])

    return x_data


def proposedmodel(number_of_unit, embedding_name, ner_limit, num_filter):
    """
    Arguments:
        number_of_unit: GRU number of hidden units
        embedding_name: name of embedding
        ner_limit: size of patient matrix
        num_filter: list of 3 CNN filter sizes

    Outputs:
        model: the proposed keras multimodal model
    """
    if embedding_name == "concat":
        input_dimension = 200
    else:
        input_dimension = 100

    # 24 hours and 104 time-series variable
    ts_input = Input(shape=(24,104),  name = "timeseries_input")

    input_embeddings = Input(shape=(ner_limit, input_dimension), name = "cnn_input")

    # 3 1D convolutional layers, with no padding
    text_conv1d = Conv1D(filters=num_filter[0], kernel_size=3,
                 padding = 'valid', strides = 1, dilation_rate=1, activation='relu',
                         kernel_initializer=GlorotUniform())(input_embeddings)

    text_conv1d = Conv1D(filters=num_filter[1], kernel_size=3,
                 padding = 'valid', strides = 1, dilation_rate=1, activation='relu',
                        kernel_initializer=GlorotUniform())(text_conv1d)

    text_conv1d = Conv1D(filters=num_filter[2], kernel_size=3,
                 padding = 'valid', strides = 1, dilation_rate=1, activation='relu',
                        kernel_initializer=GlorotUniform())(text_conv1d)

    # max pooling layer
    text_embeddings = GlobalMaxPooling1D()(text_conv1d)

    # GRU layer for time series data
    x = GRU(number_of_unit)(ts_input)

    # concatenate time series features and medical entity feature
    concatenated = Concatenate(axis=1)([x, text_embeddings])

    # Fully connected layer
    concatenated = Dense(512, activation='relu')(concatenated)
    concatenated = Dropout(0.2)(concatenated)

    preds = Dense(1, activation='sigmoid',use_bias=False,
                         kernel_initializer=GlorotUniform(),
                  kernel_regularizer=L2(l2=0.01))(concatenated)
    # define and compile model
    model = Model(inputs=[ts_input, input_embeddings], outputs=preds)
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['acc'])

    return model


In [ ]:
embedding_types = ['word2vec', 'fasttext','concat']
embedding_dict = [word2vec_embeddings, fasttext_embeddings, concatenated_embeddings]

target_problems = ['mort_hosp', 'mort_icu', 'los_3', 'los_7']

num_epoch = 50
model_patience = 3 # was 5
monitor_criteria = 'val_loss'
#monitor_criteria = 'val_acc'
batch_size = 64

filter_sizes = [32,64,96]
ner_representation_limit = 64  # tunable? # not specified in paper
activation_func = "relu"
current_model_name = "Proposed"
ts_hidden_unit = 256

maxiter = 11 #  2 for demo and checks

In [ ]:
%%time
# On GPU: 1h 37min 3s for all 3 (CPU was 5 - 6 hours or so)
# average ~ 7 seconds per epoc
# This run was on GPU

for embed_dict, embed_name in zip(embedding_dict, embedding_types):
    print ("Embedding: ", embed_name)
    print("=============================")

    # split emdeddings into training, validation and testing
    temp_train_ner = dict((k, embed_dict[k]) for k in train_ids)
    tem_dev_ner = dict((k, embed_dict[k]) for k in dev_ids)
    temp_test_ner = dict((k, embed_dict[k]) for k in test_ids)

    # combine medical entiy embeddings vertically, with padding, to create a patient matrix represention
    x_train_dict = get_subvector_data(ner_representation_limit, embed_name, temp_train_ner)
    x_dev_dict = get_subvector_data(ner_representation_limit, embed_name, tem_dev_ner)
    x_test_dict = get_subvector_data(ner_representation_limit, embed_name, temp_test_ner)

    x_train_dict_sorted = collections.OrderedDict(sorted(x_train_dict.items()))
    x_dev_dict_sorted = collections.OrderedDict(sorted(x_dev_dict.items()))
    x_test_dict_sorted = collections.OrderedDict(sorted(x_test_dict.items()))

    # convert to np array
    x_train_ner = np.array(list(x_train_dict_sorted.values()))
    x_dev_ner = np.asarray(list(x_dev_dict_sorted.values()))
    x_test_ner = np.asarray(list(x_test_dict_sorted.values()))

    del x_train_dict, x_dev_dict, x_test_dict
    del x_train_dict_sorted, x_dev_dict_sorted, x_test_dict_sorted
    gc.collect()

    for iteration in range(1,maxiter):
        print ("Iteration number: ", iteration)

        #set a different seed for each iteration
        set_seed(SEED + iteration)

        for each_problem in target_problems:
            print ("Problem type: ", each_problem)
            print ("__________________")

            # employ early stopping
            early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)

            # create checkpoint to save best model weights only (should save all 10 iterations in future)
            best_model_name = f"{PROJECT_PATH}/results/"+str(ner_representation_limit)+"-basiccnn1d-"+str(embed_name)+"-"+str(each_problem)+"-"+"best_model.hdf5"
            checkpoint = ModelCheckpoint(best_model_name, monitor=monitor_criteria, verbose=1,
                save_best_only=True, mode='min',save_weights_only=True)

            callbacks = [early_stopping_monitor, checkpoint]

            # load and train model with above callbacks
            model = proposedmodel(ts_hidden_unit,
                               embed_name, ner_representation_limit,filter_sizes)
            model.fit([x_train_ts, x_train_ner], y_train[each_problem], epochs=num_epoch, verbose=1,
                      validation_data=([x_dev_ts, x_dev_ner], y_dev[each_problem]), callbacks=callbacks, batch_size=batch_size)

            # make predictions and evaluate
            probs, predictions = make_predictions(model, [x_test_ts, x_test_ner])
            print_scores(predictions, probs, y_test[each_problem], embed_name, each_problem, iteration, ts_hidden_unit)

            # save and print the best model scores after each iteration
            model.load_weights(best_model_name)
            probs, predictions = make_predictions(model, [x_test_ts, x_test_ner])
            save_scores(predictions, probs, y_test[each_problem], embed_name, each_problem, iteration,
                            ts_hidden_unit, type_of_ner, current_model_name)
            del model
            clear_session()
            gc.collect()



Embedding:  word2vec
Iteration number:  1
Problem type:  mort_hosp
__________________


Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - ETA: 0s - loss: 0.2798 - acc: 0.9045
Epoch 1: val_loss improved from inf to 0.24145, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-basiccnn1d-word2vec-mort_hosp-best_model.hdf5
15567/15567 [==============================] - 13s 822us/sample - loss: 0.2798 - acc: 0.9045 - val_loss: 0.2414 - val_acc: 0.9147
Epoch 2/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.2373 - acc: 0.9157
Epoch 2: val_loss improved from 0.24145 to 0.22745, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-basiccnn1d-word2vec-mort_hosp-best_model.hdf5
15567/15567 [==============================] - 5s 350us/sample - loss: 0.2372 - acc: 0.9157 - val_loss: 0.2275 - val_acc: 0.9156
Epoch 3/50
15567/15567 [==============================] - ETA: 0s - loss: 0.2159 - acc: 0.9208
Epoch 3: val_loss did not impro

Problem type:  mort_icu
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15488/15567 [============================>.] - ETA: 0s - loss: 0.2131 - acc: 0.9350
Epoch 1: val_loss improved from inf to 0.17420, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-basiccnn1d-word2vec-mort_icu-best_model.hdf5
15567/15567 [==============================] - 12s 793us/sample - loss: 0.2132 - acc: 0.9351 - val_loss: 0.1742 - val_acc: 0.9431
Epoch 2/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.1700 - acc: 0.9424
Epoch 2: val_loss improved from 0.17420 to 0.16818, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-basiccnn1d-word2vec-mort_icu-best_model.hdf5
15567/15567 [==============================] - 6s 370us/sample - loss: 0.1700 - acc: 0.9424 - val_loss: 0.1682 - val_acc: 0.9436
Epoch 3/50
15567/15567 [==============================] - ETA: 0s - loss: 0.1532 - ac

Problem type:  los_3
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15488/15567 [============================>.] - ETA: 0s - loss: 0.6692 - acc: 0.6273
Epoch 1: val_loss improved from inf to 0.62236, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-basiccnn1d-word2vec-los_3-best_model.hdf5
15567/15567 [==============================] - 10s 631us/sample - loss: 0.6690 - acc: 0.6277 - val_loss: 0.6224 - val_acc: 0.6683
Epoch 2/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.6255 - acc: 0.6663
Epoch 2: val_loss improved from 0.62236 to 0.61847, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-basiccnn1d-word2vec-los_3-best_model.hdf5
15567/15567 [==============================] - 6s 388us/sample - loss: 0.6254 - acc: 0.6665 - val_loss: 0.6185 - val_acc: 0.6710
Epoch 3/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.6079 - acc: 0.6802

Problem type:  los_7
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15488/15567 [============================>.] - ETA: 0s - loss: 0.2926 - acc: 0.9178
Epoch 1: val_loss improved from inf to 0.26313, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-basiccnn1d-word2vec-los_7-best_model.hdf5
15567/15567 [==============================] - 12s 753us/sample - loss: 0.2926 - acc: 0.9177 - val_loss: 0.2631 - val_acc: 0.9237
Epoch 2/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.2521 - acc: 0.9221
Epoch 2: val_loss improved from 0.26313 to 0.25958, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-basiccnn1d-word2vec-los_7-best_model.hdf5
15567/15567 [==============================] - 5s 345us/sample - loss: 0.2520 - acc: 0.9221 - val_loss: 0.2596 - val_acc: 0.9255
Epoch 3/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.2392 - acc: 0.9225

...

Problem type:  los_7
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.2872 - acc: 0.9196
Epoch 1: val_loss improved from inf to 0.26560, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-basiccnn1d-concat-los_7-best_model.hdf5
15567/15567 [==============================] - 10s 669us/sample - loss: 0.2876 - acc: 0.9195 - val_loss: 0.2656 - val_acc: 0.9237
Epoch 2/50
15567/15567 [==============================] - ETA: 0s - loss: 0.2535 - acc: 0.9217
Epoch 2: val_loss did not improve from 0.26560
15567/15567 [==============================] - 7s 476us/sample - loss: 0.2535 - acc: 0.9217 - val_loss: 0.2715 - val_acc: 0.9237
Epoch 3/50
15567/15567 [==============================] - ETA: 0s - loss: 0.2415 - acc: 0.9228
Epoch 3: val_loss did not improve from 0.26560
15567/15567 [==============================] - 6s 392us/sample - loss: 0.2415 - acc: 0.9228 -

Iteration number:  10
Problem type:  mort_hosp
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15488/15567 [============================>.] - ETA: 0s - loss: 0.2916 - acc: 0.9023
Epoch 1: val_loss improved from inf to 0.25181, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-basiccnn1d-concat-mort_hosp-best_model.hdf5
15567/15567 [==============================] - 10s 663us/sample - loss: 0.2910 - acc: 0.9026 - val_loss: 0.2518 - val_acc: 0.9075
Epoch 2/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.2393 - acc: 0.9131
Epoch 2: val_loss improved from 0.25181 to 0.24151, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-basiccnn1d-concat-mort_hosp-best_model.hdf5
15567/15567 [==============================] - 7s 425us/sample - loss: 0.2391 - acc: 0.9132 - val_loss: 0.2415 - val_acc: 0.9174
Epoch 3/50
15488/15567 [============================>.] - ETA: 0

Problem type:  mort_icu
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15488/15567 [============================>.] - ETA: 0s - loss: 0.2131 - acc: 0.9353
Epoch 1: val_loss improved from inf to 0.17891, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-basiccnn1d-concat-mort_icu-best_model.hdf5
15567/15567 [==============================] - 9s 588us/sample - loss: 0.2127 - acc: 0.9354 - val_loss: 0.1789 - val_acc: 0.9431
Epoch 2/50
15567/15567 [==============================] - ETA: 0s - loss: 0.1751 - acc: 0.9422
Epoch 2: val_loss improved from 0.17891 to 0.17070, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-basiccnn1d-concat-mort_icu-best_model.hdf5
15567/15567 [==============================] - 7s 476us/sample - loss: 0.1751 - acc: 0.9422 - val_loss: 0.1707 - val_acc: 0.9427
Epoch 3/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.1563 - acc: 0.

Problem type:  los_3
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - ETA: 0s - loss: 0.6729 - acc: 0.6273
Epoch 1: val_loss improved from inf to 0.63340, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-basiccnn1d-concat-los_3-best_model.hdf5
15567/15567 [==============================] - 10s 657us/sample - loss: 0.6729 - acc: 0.6273 - val_loss: 0.6334 - val_acc: 0.6764
Epoch 2/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.6292 - acc: 0.6596
Epoch 2: val_loss improved from 0.63340 to 0.61980, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-basiccnn1d-concat-los_3-best_model.hdf5
15567/15567 [==============================] - 7s 467us/sample - loss: 0.6295 - acc: 0.6594 - val_loss: 0.6198 - val_acc: 0.6719
Epoch 3/50
15488/15567 [============================>.] - ETA: 0s - loss: 0.6101 - acc: 0.6796
Epo

Problem type:  los_7
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - ETA: 0s - loss: 0.2913 - acc: 0.9198
Epoch 1: val_loss improved from inf to 0.26710, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-basiccnn1d-concat-los_7-best_model.hdf5
15567/15567 [==============================] - 9s 583us/sample - loss: 0.2913 - acc: 0.9198 - val_loss: 0.2671 - val_acc: 0.9237
Epoch 2/50
15567/15567 [==============================] - ETA: 0s - loss: 0.2543 - acc: 0.9221
Epoch 2: val_loss improved from 0.26710 to 0.26303, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-basiccnn1d-concat-los_7-best_model.hdf5
15567/15567 [==============================] - 8s 484us/sample - loss: 0.2543 - acc: 0.9221 - val_loss: 0.2630 - val_acc: 0.9228
Epoch 3/50
15488/15567 [============================>.] - ETA: 0s - loss: 0.2426 - acc: 0.9223
Epoc

## No Convolution
multimodal with averaged embeddings

In [ ]:
def create_dataset(dict_of_ner):
     """
    Arguments:
        dict_of_ner: dictionary of word embeddings

    Outputs:
        average all word embeddings for each patient
    """
    temp_data = []
    for k, v in sorted(dict_of_ner.items()):
        temp = []
        for embed in v:
            temp.append(embed)
        temp_data.append(np.mean(temp, axis = 0))
    return np.asarray(temp_data)

def avg_ner_model(number_of_unit, embedding_name):
    """
    Arguments:
        number_of_unit: GRU number of hidden units
        embedding_name: name of embedding

    Outputs:
        model: the multimodal model without convolution
    """

    if embedding_name == "concat":
        input_dimension = 200
    else:
        input_dimension = 100

    # 24 hours and 104 time-series variable
    ts_input = Input(shape=(24,104),  name = "timeseries_input")

    #
    input_avg_embeddings = Input(shape=(input_dimension, ), name = "avg")

    x = GRU(number_of_unit)(ts_input)

    concatenated = Concatenate(axis=1)([x, input_avg_embeddings])

    concatenated = Dense(512, activation='relu')(concatenated)
    concatenated = Dropout(0.2)(concatenated)

    preds = Dense(1, activation='sigmoid',use_bias=False,
                         kernel_initializer=GlorotUniform(),
                  kernel_regularizer=L2(l2=0.01))(concatenated)

    model = Model(inputs=[ts_input, input_avg_embeddings], outputs=preds)
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['acc'])

    return model

In [ ]:
# 1hr 8 mins on GPU, 5 hours on CPU (22 sec per epoc average)
# This run was on CPU.
current_model_name = "No Convolution"

for embed_dict, embed_name in zip(embedding_dict, embedding_types):
    print ("Embedding: ", embed_name)
    print("=============================")

    temp_train_ner = dict((k, embed_dict[k]) for k in train_ids)
    temp_dev_ner = dict((k, embed_dict[k]) for k in dev_ids)
    temp_test_ner = dict((k, embed_dict[k]) for k in test_ids)

    x_train_ner = create_dataset(temp_train_ner)
    x_dev_ner = create_dataset(temp_dev_ner)
    x_test_ner = create_dataset(temp_test_ner)

    del temp_train_ner, temp_dev_ner, temp_test_ner


    for iteration in range(1, maxiter):
        print ("Iteration number: ", iteration)
        #set a different seed for each iteration
        set_seed(SEED + iteration)

        for each_problem in target_problems:
            print ("Problem type: ", each_problem)
            print ("__________________")

            early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)

            best_model_name = "avg-"+str(embed_name)+"-"+str(each_problem)+"-"+"best_model.hdf5"

            checkpoint = ModelCheckpoint(best_model_name, monitor=monitor_criteria, verbose=1,
                save_best_only=True, mode='min',save_weights_only=True)

            callbacks = [early_stopping_monitor, checkpoint]

            model = avg_ner_model(ts_hidden_unit, embed_name)

            model.fit([x_train_ts, x_train_ner], y_train[each_problem], epochs=num_epoch, verbose=1,
                        validation_data=([x_dev_ts, x_dev_ner], y_dev[each_problem]), callbacks=callbacks,
                        batch_size=batch_size )

            model.load_weights(best_model_name)

            probs, predictions = make_predictions(model, [x_test_ts, x_test_ner])
            print_scores(predictions, probs, y_test[each_problem], embed_name, each_problem, iteration, ts_hidden_unit)

            save_scores(predictions, probs, y_test[each_problem], embed_name, each_problem, iteration,
                            ts_hidden_unit, type_of_ner, current_model_name)

            #reset_keras(model)
            del model
            clear_session()
            gc.collect()

Embedding:  word2vec
Iteration number:  1
Problem type:  mort_hosp
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - ETA: 0s - loss: 0.2799 - acc: 0.9022
Epoch 1: val_loss improved from inf to 0.23792, saving model to avg-word2vec-mort_hosp-best_model.hdf5
15567/15567 [==============================] - 51s 3ms/sample - loss: 0.2799 - acc: 0.9022 - val_loss: 0.2379 - val_acc: 0.9116
Epoch 2/50
15567/15567 [==============================] - ETA: 0s - loss: 0.2400 - acc: 0.9142
Epoch 2: val_loss improved from 0.23792 to 0.22367, saving model to avg-word2vec-mort_hosp-best_model.hdf5
15567/15567 [==============================] - 33s 2ms/sample - loss: 0.2400 - acc: 0.9142 - val_loss: 0.2237 - val_acc: 0.9183
Epoch 3/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.2204 - acc: 0.9212
Epoch 3: val_loss did not improve from 0.22367
15567/15567 [==============================] - 27s 2ms/sample - los

## Time Series/No Medical Entities

In [ ]:
def timeseries_model(number_of_unit):
    """
    Arguments:
        number_of_unit: GRU number of hidden units

    Outputs:
        model: unimodal time series model from proposed model
    """
    K.clear_session()

    ts_input = Input(shape=(24,104),  name = "timeseries_input")

    x = GRU(number_of_unit)(ts_input)

    preds = Dense(1, activation='sigmoid',use_bias=False,
                         kernel_initializer=GlorotUniform(),
                  kernel_regularizer=L2(l2=0.01))(x)

    model = Model(inputs=ts_input, outputs=preds)

    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['acc'])
    return model

In [ ]:
%%time
# On GPU: 1723.633sec/ 28min 43s (CPU may be 1 hour or so)
# 6 seconds per epoc average
# This run was on GPU

current_model_name = "Time Series" # No Medical Entities

for iteration in range(1, maxiter):
    print("Iteration number: ", iteration)
    print("=============================")
    set_seed(SEED + iteration)

    for each_problem in target_problems:
        print ("Problem type: ", each_problem)
        print ("__________________")


        early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)

        best_model_name = f"{PROJECT_PATH}/results/GRU-"+str(each_problem)+"-"+"best_model.hdf5"

        checkpoint = ModelCheckpoint(best_model_name, monitor=monitor_criteria, verbose=1,
            save_best_only=True, mode='min',save_weights_only=True)


        callbacks = [early_stopping_monitor, checkpoint]

        model = timeseries_model(ts_hidden_unit)
        model.fit(x_train_ts, y_train[each_problem], epochs=num_epoch, verbose=1,
                  validation_data=(x_dev_ts, y_dev[each_problem]), callbacks=callbacks, batch_size= batch_size)

        model.load_weights(best_model_name)

        probs, predictions = make_predictions(model, x_test_ts)
        save_scores(predictions, probs, y_test[each_problem].values,'N/A', each_problem,
                                iteration, ts_hidden_unit, type_of_ner,current_model_name)
        #reset_keras(model)
        del model
        clear_session()
        gc.collect()

Iteration number:  1
Problem type:  mort_hosp
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15488/15567 [============================>.] - ETA: 0s - loss: 0.2872 - acc: 0.8977
Epoch 1: val_loss improved from inf to 0.24833, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/GRU-mort_hosp-best_model.hdf5
15567/15567 [==============================] - 8s 491us/sample - loss: 0.2874 - acc: 0.8977 - val_loss: 0.2483 - val_acc: 0.9174
Epoch 2/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.2417 - acc: 0.9144
Epoch 2: val_loss improved from 0.24833 to 0.22883, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/GRU-mort_hosp-best_model.hdf5
15567/15567 [==============================] - 4s 289us/sample - loss: 0.2417 - acc: 0.9144 - val_loss: 0.2288 - val_acc: 0.9161
Epoch 3/50
15488/15567 [============================>.] - ETA: 0s - loss: 0.2258 - acc: 0.9180
Epoch

Problem type:  mort_icu
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15488/15567 [============================>.] - ETA: 0s - loss: 0.2267 - acc: 0.9241
Epoch 1: val_loss improved from inf to 0.17828, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/GRU-mort_icu-best_model.hdf5
15567/15567 [==============================] - 8s 516us/sample - loss: 0.2268 - acc: 0.9241 - val_loss: 0.1783 - val_acc: 0.9436
Epoch 2/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.1777 - acc: 0.9410
Epoch 2: val_loss improved from 0.17828 to 0.16695, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/GRU-mort_icu-best_model.hdf5
15567/15567 [==============================] - 5s 291us/sample - loss: 0.1777 - acc: 0.9410 - val_loss: 0.1669 - val_acc: 0.9440
Epoch 3/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.1631 - acc: 0.9453
Epoch 3: val_loss improved fr

Problem type:  los_3
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - ETA: 0s - loss: 0.6570 - acc: 0.6376
Epoch 1: val_loss improved from inf to 0.63151, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/GRU-los_3-best_model.hdf5
15567/15567 [==============================] - 8s 530us/sample - loss: 0.6570 - acc: 0.6376 - val_loss: 0.6315 - val_acc: 0.6715
Epoch 2/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.6305 - acc: 0.6612
Epoch 2: val_loss improved from 0.63151 to 0.62121, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/GRU-los_3-best_model.hdf5
15567/15567 [==============================] - 4s 284us/sample - loss: 0.6304 - acc: 0.6613 - val_loss: 0.6212 - val_acc: 0.6715
Epoch 3/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.6168 - acc: 0.6735
Epoch 3: val_loss improved from 0.6212

Problem type:  los_7
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.2984 - acc: 0.9086
Epoch 1: val_loss improved from inf to 0.26022, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/GRU-los_7-best_model.hdf5
15567/15567 [==============================] - 7s 431us/sample - loss: 0.2982 - acc: 0.9087 - val_loss: 0.2602 - val_acc: 0.9246
Epoch 2/50
15567/15567 [==============================] - ETA: 0s - loss: 0.2564 - acc: 0.9219
Epoch 2: val_loss improved from 0.26022 to 0.25494, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/GRU-los_7-best_model.hdf5
15567/15567 [==============================] - 5s 328us/sample - loss: 0.2564 - acc: 0.9219 - val_loss: 0.2549 - val_acc: 0.9242
Epoch 3/50
15488/15567 [============================>.] - ETA: 0s - loss: 0.2455 - acc: 0.9226
Epoch 3: val_loss improved from 0.2549

Iteration number:  2
Problem type:  mort_hosp
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - ETA: 0s - loss: 0.2904 - acc: 0.8989
Epoch 1: val_loss improved from inf to 0.24273, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/GRU-mort_hosp-best_model.hdf5
15567/15567 [==============================] - 7s 454us/sample - loss: 0.2904 - acc: 0.8989 - val_loss: 0.2427 - val_acc: 0.9152
Epoch 2/50
15488/15567 [============================>.] - ETA: 0s - loss: 0.2394 - acc: 0.9147
Epoch 2: val_loss improved from 0.24273 to 0.24019, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/GRU-mort_hosp-best_model.hdf5
15567/15567 [==============================] - 5s 291us/sample - loss: 0.2395 - acc: 0.9148 - val_loss: 0.2402 - val_acc: 0.9088
Epoch 3/50
15424/15567 [============================>.] - ETA: 0s - loss: 0.2263 - acc: 0.9194
Epoch

...

Problem type:  los_7
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.3027 - acc: 0.9080
Epoch 1: val_loss improved from inf to 0.26910, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/GRU-los_7-best_model.hdf5
15567/15567 [==============================] - 7s 477us/sample - loss: 0.3025 - acc: 0.9081 - val_loss: 0.2691 - val_acc: 0.9242
Epoch 2/50
15567/15567 [==============================] - ETA: 0s - loss: 0.2560 - acc: 0.9225
Epoch 2: val_loss improved from 0.26910 to 0.25392, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/GRU-los_7-best_model.hdf5
15567/15567 [==============================] - 5s 297us/sample - loss: 0.2560 - acc: 0.9225 - val_loss: 0.2539 - val_acc: 0.9228
Epoch 3/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.2453 - acc: 0.9223
Epoch 3: val_loss did not improve from

Iteration number:  10
Problem type:  mort_hosp
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15488/15567 [============================>.] - ETA: 0s - loss: 0.2917 - acc: 0.8979
Epoch 1: val_loss improved from inf to 0.25343, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/GRU-mort_hosp-best_model.hdf5
15567/15567 [==============================] - 11s 724us/sample - loss: 0.2912 - acc: 0.8981 - val_loss: 0.2534 - val_acc: 0.9106
Epoch 2/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.2422 - acc: 0.9120
Epoch 2: val_loss improved from 0.25343 to 0.23762, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/GRU-mort_hosp-best_model.hdf5
15567/15567 [==============================] - 8s 511us/sample - loss: 0.2421 - acc: 0.9121 - val_loss: 0.2376 - val_acc: 0.9156
Epoch 3/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.2257 - acc: 0.9193
Epo

Problem type:  mort_icu
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.2278 - acc: 0.9244
Epoch 1: val_loss improved from inf to 0.17968, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/GRU-mort_icu-best_model.hdf5
15567/15567 [==============================] - 7s 456us/sample - loss: 0.2276 - acc: 0.9245 - val_loss: 0.1797 - val_acc: 0.9431
Epoch 2/50
15488/15567 [============================>.] - ETA: 0s - loss: 0.1768 - acc: 0.9426
Epoch 2: val_loss improved from 0.17968 to 0.17223, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/GRU-mort_icu-best_model.hdf5
15567/15567 [==============================] - 5s 298us/sample - loss: 0.1766 - acc: 0.9427 - val_loss: 0.1722 - val_acc: 0.9422
Epoch 3/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.1622 - acc: 0.9445
Epoch 3: val_loss improved fr

Problem type:  los_3
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.6587 - acc: 0.6381
Epoch 1: val_loss improved from inf to 0.63505, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/GRU-los_3-best_model.hdf5
15567/15567 [==============================] - 7s 467us/sample - loss: 0.6587 - acc: 0.6382 - val_loss: 0.6350 - val_acc: 0.6715
Epoch 2/50
15567/15567 [==============================] - ETA: 0s - loss: 0.6332 - acc: 0.6621
Epoch 2: val_loss improved from 0.63505 to 0.62044, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/GRU-los_3-best_model.hdf5
15567/15567 [==============================] - 6s 356us/sample - loss: 0.6332 - acc: 0.6621 - val_loss: 0.6204 - val_acc: 0.6801
Epoch 3/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.6201 - acc: 0.6692
Epoch 3: val_loss improved from 0.6204

Problem type:  los_7
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - ETA: 0s - loss: 0.3044 - acc: 0.9066
Epoch 1: val_loss improved from inf to 0.27030, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/GRU-los_7-best_model.hdf5
15567/15567 [==============================] - 7s 466us/sample - loss: 0.3044 - acc: 0.9066 - val_loss: 0.2703 - val_acc: 0.9188
Epoch 2/50
15567/15567 [==============================] - ETA: 0s - loss: 0.2564 - acc: 0.9216
Epoch 2: val_loss improved from 0.27030 to 0.25769, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/GRU-los_7-best_model.hdf5
15567/15567 [==============================] - 5s 300us/sample - loss: 0.2564 - acc: 0.9216 - val_loss: 0.2577 - val_acc: 0.9233
Epoch 3/50
15567/15567 [==============================] - ETA: 0s - loss: 0.2465 - acc: 0.9225
Epoch 3: val_loss improved from 0.2576

## No Time Series

In [ ]:
def avg_ner_only_model(embedding_name):
    """
    Arguments:
        embedding_name: name of embedding

    Outputs:
        model: averaged medical entity model
    """
    if embedding_name == "concat":
        input_dimension = 200
    else:
        input_dimension = 100


    input_avg_embeddings = Input(shape=(input_dimension, ), name = "avg")

    x = Dense(512, activation='relu')(input_avg_embeddings)
    x = Dropout(0.2)(x)

    preds = Dense(1, activation='sigmoid',use_bias=False,
                         kernel_initializer=GlorotUniform(),
                  kernel_regularizer=L2(l2=0.01))(x)

    model = Model(inputs=[input_avg_embeddings], outputs=preds)
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['acc'])

    return model

In [ ]:
%%time
current_model_name = "No Time Series"

for embed_dict, embed_name in zip(embedding_dict, embedding_types):
    print ("Embedding: ", embed_name)
    print("=============================")

    temp_train_ner = dict((k, embed_dict[k]) for k in train_ids)
    temp_dev_ner = dict((k, embed_dict[k]) for k in dev_ids)
    temp_test_ner = dict((k, embed_dict[k]) for k in test_ids)

    x_train_ner = create_dataset(temp_train_ner)
    x_dev_ner = create_dataset(temp_dev_ner)
    x_test_ner = create_dataset(temp_test_ner)

    del temp_train_ner, temp_dev_ner, temp_test_ner


    for iteration in range(1, maxiter):
        print ("Iteration number: ", iteration)
        #set a different seed for each iteration
        set_seed(SEED + iteration)

        for each_problem in target_problems:
            print ("Problem type: ", each_problem)
            print ("__________________")

            early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)

            best_model_name = "avg-ner-only-"+str(embed_name)+"-"+str(each_problem)+"-"+".hdf5"

            checkpoint = ModelCheckpoint(best_model_name, monitor=monitor_criteria, verbose=1,
                save_best_only=True, mode='min',save_weights_only=True)

            callbacks = [early_stopping_monitor, checkpoint]

            model = avg_ner_only_model(embed_name)

            model.fit(x_train_ner, y_train[each_problem], epochs=num_epoch, verbose=1,
                        validation_data=(x_dev_ner, y_dev[each_problem]), callbacks=callbacks,
                        batch_size=batch_size )

            model.load_weights(best_model_name)

            probs, predictions = make_predictions(model, x_test_ner)
            print_scores(predictions, probs, y_test[each_problem], embed_name, each_problem, iteration, ts_hidden_unit)

            save_scores(predictions, probs, y_test[each_problem], embed_name, each_problem, iteration,
                            ts_hidden_unit, type_of_ner, current_model_name)

            #reset_keras(model)
            del model
            clear_session()
            gc.collect()

Streaming output truncated to the last 5000 lines.
Epoch 9: val_loss did not improve from 0.22085
15567/15567 [==============================] - 1s 62us/sample - loss: 0.2207 - acc: 0.9316 - val_loss: 0.2214 - val_acc: 0.9355
Epoch 10/50
14848/15567 [===========================>..] - ETA: 0s - loss: 0.2172 - acc: 0.9320
Epoch 10: val_loss did not improve from 0.22085
15567/15567 [==============================] - 1s 60us/sample - loss: 0.2194 - acc: 0.9310 - val_loss: 0.2288 - val_acc: 0.9337
Epoch 11/50
15424/15567 [============================>.] - ETA: 0s - loss: 0.2189 - acc: 0.9314
Epoch 11: val_loss improved from 0.22085 to 0.22080, saving model to avg-ner-only-word2vec-mort_icu-.hdf5
15567/15567 [==============================] - 1s 62us/sample - loss: 0.2184 - acc: 0.9318 - val_loss: 0.2208 - val_acc: 0.9355
Epoch 12/50
15360/15567 [============================>.] - ETA: 0s - loss: 0.2164 - acc: 0.9316
Epoch 12: val_loss improved from 0.22080 to 0.21830, saving model to avg-ner

# Hyperparameter tuning

Ran model with different parameters and manually chose the one with the lowest validation loss of 0.223

In [ ]:
def proposedmodel(number_of_unit, embedding_name, ner_limit, num_filter, learning_rate,dropout,l2_norm):
    if embedding_name == "concat":
        input_dimension = 200
    else:
        input_dimension = 100

    # 24 hours and 104 time-series variable
    ts_input = Input(shape=(24,104),  name = "timeseries_input")

    input_embeddings = Input(shape=(ner_limit, input_dimension), name = "cnn_input")


    text_conv1d = Conv1D(filters=num_filter[0], kernel_size=3,
                 padding = 'valid', strides = 1, dilation_rate=1, activation='relu',
                         kernel_initializer=GlorotUniform())(input_embeddings)

    text_conv1d = Conv1D(filters=num_filter[1], kernel_size=3,
                 padding = 'valid', strides = 1, dilation_rate=1, activation='relu',
                        kernel_initializer=GlorotUniform())(text_conv1d)

    text_conv1d = Conv1D(filters=num_filter[2], kernel_size=3,
                 padding = 'valid', strides = 1, dilation_rate=1, activation='relu',
                        kernel_initializer=GlorotUniform())(text_conv1d)


    text_embeddings = GlobalMaxPooling1D()(text_conv1d)

    x = GRU(number_of_unit)(ts_input)

    concatenated = Concatenate(axis=1)([x, text_embeddings])

    concatenated = Dense(512, activation='relu')(concatenated)
    concatenated = Dropout(dropout)(concatenated)

    preds = Dense(1, activation='sigmoid',use_bias=False,
                         kernel_initializer=GlorotUniform(),
                  kernel_regularizer=L2(l2=l2_norm))(concatenated)

    model = Model(inputs=[ts_input, input_embeddings], outputs=preds)
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=learning_rate),
                  metrics=['acc'])

    return model

In [ ]:
# only using Word2Vec and In hospital mortality for tunning
embedding_types = ['word2vec']
embedding_dict = [word2vec_embeddings]

target_problems = ['mort_hosp']

num_epoch = 50
model_patience = 3 # was 5
monitor_criteria = 'val_loss'
batch_size = 64

filter_sizes = [32,64,96]
ner_representation_limit = 64  # (tried to tune earlier but 64 was the best)
ts_hidden_unit = 256 # number of units

maxiter = 2


In [ ]:
# more than 2.5 hours on GPU and it disconnected so needed to run the remainder in next cell
""""
From paper: we independently tune the hyperparameters – number of hidden layers, hidden units, convolutional
filters, filter size, learning rate, dropout rates and regularization parameters on the validation set
"""
# Define parameters for tuning
number_of_unit_range = [128, 256] # original 256
num_filter_range = [(32, 64, 96), (64, 96, 128), [96, 128, 150]] # original (32, 64, 96)
learning_rate_range = [0.0005, 0.001] # original 0.001
dropout_rate_range = [0.1, 0.2, 0.3, 0.4, 0.5] # original 0.2
l2_regularization_range = [0.01, 0.05, 0.1] # original 0.01

for number_of_unit, filter_sizes, learning_rate, dropout, l2_reg in itertools.product(number_of_unit_range, num_filter_range,learning_rate_range,dropout_rate_range,l2_regularization_range):
    print("Number of units:", number_of_unit)
    print("Filter sizes:", filter_sizes)
    print("Learning rate:", learning_rate)
    print("Dropout:", dropout)
    print("Regularizer:", l2_reg)
    print("=============================")

    for embed_dict, embed_name in zip(embedding_dict, embedding_types):
        print("Embedding:", embed_name)
        print("=============================")

        # Your data preparation code here
        # Assuming x_train_ts, x_train_ner, y_train, etc. are defined somewhere

        temp_train_ner = dict((k, embed_dict[k]) for k in train_ids)
        tem_dev_ner = dict((k, embed_dict[k]) for k in dev_ids)
        temp_test_ner = dict((k, embed_dict[k]) for k in test_ids)

        x_train_dict = get_subvector_data(ner_representation_limit, embed_name, temp_train_ner)
        x_dev_dict = get_subvector_data(ner_representation_limit, embed_name, tem_dev_ner)
        x_test_dict = get_subvector_data(ner_representation_limit, embed_name, temp_test_ner)

        x_train_dict_sorted = collections.OrderedDict(sorted(x_train_dict.items()))
        x_dev_dict_sorted = collections.OrderedDict(sorted(x_dev_dict.items()))
        x_test_dict_sorted = collections.OrderedDict(sorted(x_test_dict.items()))

        # convert to np array
        x_train_ner = np.array(list(x_train_dict_sorted.values()))
        x_dev_ner = np.asarray(list(x_dev_dict_sorted.values()))
        x_test_ner = np.asarray(list(x_test_dict_sorted.values()))

        del x_train_dict, x_dev_dict, x_test_dict
        del x_train_dict_sorted, x_dev_dict_sorted, x_test_dict_sorted
        gc.collect()

        for iteration in range(1, maxiter):
            print("Iteration number:", iteration)

             #set a different seed for each iteration
            set_seed(SEED + iteration)

            for each_problem in target_problems:
                print ("Problem type: ", each_problem)
                print ("__________________")

                early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)

                #
                model = proposedmodel(number_of_unit, embed_name, ner_representation_limit, filter_sizes,learning_rate,dropout,l2_reg)
                model.fit([x_train_ts, x_train_ner], y_train[each_problem], epochs=num_epoch, verbose=1,
                        validation_data=([x_dev_ts, x_dev_ner], y_dev[each_problem]), callbacks=[early_stopping_monitor], batch_size=batch_size)


                del model
                clear_session()
                gc.collect()


Number of units: 128
Filter sizes: (32, 64, 96)
Learning rate: 0.0005
Dropout: 0.1
Regularizer: 0.01
Embedding: word2vec


Iteration number: 1
Problem type:  mort_hosp
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - 7s 423us/sample - loss: 0.2880 - acc: 0.9045 - val_loss: 0.2460 - val_acc: 0.9143
Epoch 2/50
15567/15567 [==============================] - 5s 323us/sample - loss: 0.2417 - acc: 0.9146 - val_loss: 0.2376 - val_acc: 0.9147
Epoch 3/50
15567/15567 [==============================] - 6s 393us/sample - loss: 0.2164 - acc: 0.9229 - val_loss: 0.2375 - val_acc: 0.9125
Epoch 4/50
15567/15567 [==============================] - 5s 322us/sample - loss: 0.1909 - acc: 0.9306 - val_loss: 0.2488 - val_acc: 0.9061
Epoch 5/50
15567/15567 [==============================] - 5s 325us/sample - loss: 0.1600 - acc: 0.9426 - val_loss: 0.2674 - val_acc: 0.9025
Epoch 6/50
15567/15567 [==============================] - 6s 396us/sample - loss: 0.1225 - acc: 0.9566 - val_loss: 0.3052 - val_acc: 0.8989
Number of units: 128
Filter sizes: (32, 64, 96)

Iteration number: 1
Problem type:  mort_hosp
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - 8s 545us/sample - loss: 0.3252 - acc: 0.9047 - val_loss: 0.2644 - val_acc: 0.9152
Epoch 2/50
15567/15567 [==============================] - 5s 322us/sample - loss: 0.2524 - acc: 0.9149 - val_loss: 0.2415 - val_acc: 0.9161
Epoch 3/50
15567/15567 [==============================] - 6s 373us/sample - loss: 0.2232 - acc: 0.9217 - val_loss: 0.2382 - val_acc: 0.9111
Epoch 4/50
15567/15567 [==============================] - 5s 341us/sample - loss: 0.2004 - acc: 0.9285 - val_loss: 0.2446 - val_acc: 0.9052
Epoch 5/50
15567/15567 [==============================] - 5s 314us/sample - loss: 0.1736 - acc: 0.9383 - val_loss: 0.2494 - val_acc: 0.9120
Epoch 6/50
15567/15567 [==============================] - 6s 397us/sample - loss: 0.1351 - acc: 0.9542 - val_loss: 0.2920 - val_acc: 0.9057
Number of units: 128
Filter sizes: (32, 64, 96)

Iteration number: 1
Problem type:  mort_hosp
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - 9s 554us/sample - loss: 0.3587 - acc: 0.9043 - val_loss: 0.2700 - val_acc: 0.9147
Epoch 2/50
15567/15567 [==============================] - 5s 352us/sample - loss: 0.2549 - acc: 0.9143 - val_loss: 0.2404 - val_acc: 0.9143
Epoch 3/50
15567/15567 [==============================] - 6s 360us/sample - loss: 0.2258 - acc: 0.9211 - val_loss: 0.2360 - val_acc: 0.9129
Epoch 4/50
15567/15567 [==============================] - 6s 354us/sample - loss: 0.2061 - acc: 0.9264 - val_loss: 0.2394 - val_acc: 0.9111
Epoch 5/50
15567/15567 [==============================] - 5s 320us/sample - loss: 0.1845 - acc: 0.9336 - val_loss: 0.2449 - val_acc: 0.9061
Epoch 6/50
15567/15567 [==============================] - 6s 395us/sample - loss: 0.1492 - acc: 0.9471 - val_loss: 0.2778 - val_acc: 0.9012
Number of units: 128
Filter sizes: (32, 64, 96)

Iteration number: 1
Problem type:  mort_hosp
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - 9s 552us/sample - loss: 0.2902 - acc: 0.9040 - val_loss: 0.2478 - val_acc: 0.9134
Epoch 2/50
15567/15567 [==============================] - 5s 320us/sample - loss: 0.2430 - acc: 0.9151 - val_loss: 0.2377 - val_acc: 0.9170
Epoch 3/50
15567/15567 [==============================] - 5s 348us/sample - loss: 0.2177 - acc: 0.9220 - val_loss: 0.2363 - val_acc: 0.9102
Epoch 4/50
15567/15567 [==============================] - 6s 368us/sample - loss: 0.1940 - acc: 0.9308 - val_loss: 0.2481 - val_acc: 0.9079
Epoch 5/50
15567/15567 [==============================] - 5s 347us/sample - loss: 0.1642 - acc: 0.9422 - val_loss: 0.2553 - val_acc: 0.9134
Epoch 6/50
15567/15567 [==============================] - 6s 399us/sample - loss: 0.1255 - acc: 0.9565 - val_loss: 0.3275 - val_acc: 0.8858
Number of units: 128
Filter sizes: (32, 64, 96)

...

Iteration number: 1
Problem type:  mort_hosp
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - 9s 582us/sample - loss: 0.3234 - acc: 0.9043 - val_loss: 0.2576 - val_acc: 0.9152
Epoch 2/50
15567/15567 [==============================] - 6s 354us/sample - loss: 0.2497 - acc: 0.9137 - val_loss: 0.2347 - val_acc: 0.9152
Epoch 3/50
15567/15567 [==============================] - 6s 404us/sample - loss: 0.2239 - acc: 0.9189 - val_loss: 0.2322 - val_acc: 0.9134
Epoch 4/50
15567/15567 [==============================] - 6s 382us/sample - loss: 0.2022 - acc: 0.9249 - val_loss: 0.2348 - val_acc: 0.9093
Epoch 5/50
15567/15567 [==============================] - 6s 363us/sample - loss: 0.1788 - acc: 0.9341 - val_loss: 0.2294 - val_acc: 0.9129
Epoch 6/50
15567/15567 [==============================] - 7s 425us/sample - loss: 0.1333 - acc: 0.9521 - val_loss: 0.2694 - val_acc: 0.9134
Epoch 7/50
15567/15567 [=======================

Iteration number: 1
Problem type:  mort_hosp
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - 9s 595us/sample - loss: 0.3552 - acc: 0.9040 - val_loss: 0.2615 - val_acc: 0.9161
Epoch 2/50
15567/15567 [==============================] - 5s 344us/sample - loss: 0.2507 - acc: 0.9148 - val_loss: 0.2349 - val_acc: 0.9156
Epoch 3/50
15567/15567 [==============================] - 7s 435us/sample - loss: 0.2256 - acc: 0.9183 - val_loss: 0.2309 - val_acc: 0.9134
Epoch 4/50
15567/15567 [==============================] - 5s 349us/sample - loss: 0.2060 - acc: 0.9243 - val_loss: 0.2335 - val_acc: 0.9093
Epoch 5/50
15567/15567 [==============================] - 7s 435us/sample - loss: 0.1841 - acc: 0.9338 - val_loss: 0.2317 - val_acc: 0.9102
Epoch 6/50
15567/15567 [==============================] - 6s 354us/sample - loss: 0.1476 - acc: 0.9464 - val_loss: 0.2622 - val_acc: 0.9129
Number of units: 256
Filter sizes: [96, 128, 15

Iteration number: 1
Problem type:  mort_hosp
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - 8s 531us/sample - loss: 0.2915 - acc: 0.9031 - val_loss: 0.2445 - val_acc: 0.9138
Epoch 2/50
15567/15567 [==============================] - 7s 433us/sample - loss: 0.2446 - acc: 0.9148 - val_loss: 0.2318 - val_acc: 0.9170
Epoch 3/50
15567/15567 [==============================] - 5s 351us/sample - loss: 0.2225 - acc: 0.9200 - val_loss: 0.2316 - val_acc: 0.9134
Epoch 4/50
15567/15567 [==============================] - 6s 386us/sample - loss: 0.1988 - acc: 0.9268 - val_loss: 0.2379 - val_acc: 0.9079
Epoch 5/50
15567/15567 [==============================] - 6s 399us/sample - loss: 0.1743 - acc: 0.9354 - val_loss: 0.2413 - val_acc: 0.9097
Epoch 6/50
15567/15567 [==============================] - 5s 349us/sample - loss: 0.1347 - acc: 0.9506 - val_loss: 0.2720 - val_acc: 0.8985
Number of units: 256
Filter sizes: [96, 128, 15

Iteration number: 1
Problem type:  mort_hosp
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - 9s 563us/sample - loss: 0.3260 - acc: 0.9033 - val_loss: 0.2591 - val_acc: 0.9156
Epoch 2/50
15567/15567 [==============================] - 6s 382us/sample - loss: 0.2527 - acc: 0.9145 - val_loss: 0.2361 - val_acc: 0.9152
Epoch 3/50
15567/15567 [==============================] - 6s 355us/sample - loss: 0.2263 - acc: 0.9185 - val_loss: 0.2325 - val_acc: 0.9129
Epoch 4/50
15567/15567 [==============================] - 7s 435us/sample - loss: 0.2038 - acc: 0.9252 - val_loss: 0.2360 - val_acc: 0.9084
Epoch 5/50
15567/15567 [==============================] - 6s 358us/sample - loss: 0.1832 - acc: 0.9327 - val_loss: 0.2305 - val_acc: 0.9134
Epoch 6/50
15567/15567 [==============================] - 7s 434us/sample - loss: 0.1438 - acc: 0.9480 - val_loss: 0.2707 - val_acc: 0.8953
Epoch 7/50
15567/15567 [=======================

Iteration number: 1
Problem type:  mort_hosp
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - 10s 612us/sample - loss: 0.3585 - acc: 0.9032 - val_loss: 0.2639 - val_acc: 0.9161
Epoch 2/50
15567/15567 [==============================] - 5s 351us/sample - loss: 0.2535 - acc: 0.9137 - val_loss: 0.2347 - val_acc: 0.9161
Epoch 3/50
 6080/15567 [==========>...................] - ETA: 3s - loss: 0.2184 - acc: 0.9187

In [ ]:
# # more than 2.5 hours on GPU
# """"
# From paper: we independently tune the hyperparameters – number of hidden layers, hidden units, convolutional
# filters, filter size, learning rate, dropout rates and regularization parameters on the validation set
# """
# # Define parameters for grid search
# number_of_unit_range = [256]
# num_filter_range = [[96, 128, 150]]
# learning_rate_range = [0.001] # 0.001
# dropout_rate_range = [0.1, 0.2, 0.3, 0.4, 0.5] #0.2
# l2_regularization_range = [0.01, 0.05, 0.1] #0.01

# for number_of_unit, filter_sizes,learning_rate,dropout,l2_reg  in itertools.product(number_of_unit_range, num_filter_range,learning_rate_range,dropout_rate_range,l2_regularization_range):
#     print("Number of units:", number_of_unit)
#     print("Filter sizes:", filter_sizes)
#     print("Learning rate:", learning_rate)
#     print("Dropout:", dropout)
#     print("Regularizer:", l2_reg)
#     print("=============================")

#     for embed_dict, embed_name in zip(embedding_dict, embedding_types):
#         print("Embedding:", embed_name)
#         print("=============================")

#         # Your data preparation code here
#         # Assuming x_train_ts, x_train_ner, y_train, etc. are defined somewhere

#         temp_train_ner = dict((k, embed_dict[k]) for k in train_ids)
#         tem_dev_ner = dict((k, embed_dict[k]) for k in dev_ids)
#         temp_test_ner = dict((k, embed_dict[k]) for k in test_ids)

#         x_train_dict = get_subvector_data(ner_representation_limit, embed_name, temp_train_ner)
#         x_dev_dict = get_subvector_data(ner_representation_limit, embed_name, tem_dev_ner)
#         x_test_dict = get_subvector_data(ner_representation_limit, embed_name, temp_test_ner)

#         x_train_dict_sorted = collections.OrderedDict(sorted(x_train_dict.items()))
#         x_dev_dict_sorted = collections.OrderedDict(sorted(x_dev_dict.items()))
#         x_test_dict_sorted = collections.OrderedDict(sorted(x_test_dict.items()))

#         # convert to np array
#         x_train_ner = np.array(list(x_train_dict_sorted.values()))
#         x_dev_ner = np.asarray(list(x_dev_dict_sorted.values()))
#         x_test_ner = np.asarray(list(x_test_dict_sorted.values()))

#         del x_train_dict, x_dev_dict, x_test_dict
#         del x_train_dict_sorted, x_dev_dict_sorted, x_test_dict_sorted
#         gc.collect()

#         for iteration in range(1, maxiter):
#             print("Iteration number:", iteration)

#              #set a different seed for each iteration
#             set_seed(SEED + iteration)

#             for each_problem in target_problems:
#                 print ("Problem type: ", each_problem)
#                 print ("__________________")

#                 early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)

#                 #
#                 model = proposedmodel(number_of_unit, embed_name, ner_representation_limit, filter_sizes,learning_rate,dropout,l2_reg)
#                 model.fit([x_train_ts, x_train_ner], y_train[each_problem], epochs=num_epoch, verbose=1,
#                         validation_data=([x_dev_ts, x_dev_ner], y_dev[each_problem]), callbacks=[early_stopping_monitor], batch_size=batch_size)


#                 del model
#                 clear_session()
#                 gc.collect()

Number of units: 256
Filter sizes: [96, 128, 150]
Learning rate: 0.001
Dropout: 0.1
Regularizer: 0.01
Embedding: word2vec
Iteration number: 1
Problem type:  mort_hosp
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - 44s 3ms/sample - loss: 0.2791 - acc: 0.9047 - val_loss: 0.2392 - val_acc: 0.9134
Epoch 2/50
15567/15567 [==============================] - 40s 3ms/sample - loss: 0.2362 - acc: 0.9155 - val_loss: 0.2258 - val_acc: 0.9165
Epoch 3/50
15567/15567 [==============================] - 40s 3ms/sample - loss: 0.2155 - acc: 0.9218 - val_loss: 0.2325 - val_acc: 0.9125
Epoch 4/50
15567/15567 [==============================] - 40s 3ms/sample - loss: 0.1969 - acc: 0.9271 - val_loss: 0.2452 - val_acc: 0.8998
Epoch 5/50
15567/15567 [==============================] - 41s 3ms/sample - loss: 0.1727 - acc: 0.9343 - val_loss: 0.2352 - val_acc: 0.9116
Number of units: 256
Filter sizes: [96, 128, 150]
Learning rate: 0.001

## Proposed (w Tuning)

In [ ]:
def proposedmodel(number_of_unit, embedding_name, ner_limit, num_filter):
    if embedding_name == "concat":
        input_dimension = 200
    else:
        input_dimension = 100

    # 24 hours and 104 time-series variable
    ts_input = Input(shape=(24,104),  name = "timeseries_input")

    input_embeddings = Input(shape=(ner_limit, input_dimension), name = "cnn_input")


    text_conv1d = Conv1D(filters=num_filter[0], kernel_size=3,
                 padding = 'valid', strides = 1, dilation_rate=1, activation='relu',
                         kernel_initializer=GlorotUniform())(input_embeddings)

    text_conv1d = Conv1D(filters=num_filter[1], kernel_size=3,
                 padding = 'valid', strides = 1, dilation_rate=1, activation='relu',
                        kernel_initializer=GlorotUniform())(text_conv1d)

    text_conv1d = Conv1D(filters=num_filter[2], kernel_size=3,
                 padding = 'valid', strides = 1, dilation_rate=1, activation='relu',
                        kernel_initializer=GlorotUniform())(text_conv1d)


    text_embeddings = GlobalMaxPooling1D()(text_conv1d)

    x = GRU(number_of_unit)(ts_input)

    concatenated = Concatenate(axis=1)([x, text_embeddings])

    concatenated = Dense(512, activation='relu')(concatenated)
    concatenated = Dropout(0.5)(concatenated)

    preds = Dense(1, activation='sigmoid',use_bias=False,
                         kernel_initializer=GlorotUniform(),
                  kernel_regularizer=L2(l2=0.1))(concatenated)

    model = Model(inputs=[ts_input, input_embeddings], outputs=preds)
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['acc'])

    return model


In [ ]:
embedding_types = ['word2vec', 'fasttext','concat']
embedding_dict = [word2vec_embeddings, fasttext_embeddings, concatenated_embeddings]

target_problems = ['mort_hosp', 'mort_icu', 'los_3', 'los_7']

num_epoch = 50
model_patience = 3 # was 5
monitor_criteria = 'val_loss'
#monitor_criteria = 'val_acc'
batch_size = 64

filter_sizes = [64,96,128]
ner_representation_limit = 64
activation_func = "relu"
current_model_name = "Proposed (Tuned)"
ts_hidden_unit = 256

maxiter = 11 #  2 for demo and checks

In [ ]:
%%time
# 1hr 13 mins on GPU for 2 word embeddings, 4+ hours on CPU, for 2 embeddings (last experiement done,
# runtime kept disconnecting in Colab but Word2Vec and FastText were complete, missing combined)
# This run was on GPU

for embed_dict, embed_name in zip(embedding_dict, embedding_types):
    print ("Embedding: ", embed_name)
    print("=============================")

    temp_train_ner = dict((k, embed_dict[k]) for k in train_ids)
    tem_dev_ner = dict((k, embed_dict[k]) for k in dev_ids)
    temp_test_ner = dict((k, embed_dict[k]) for k in test_ids)

    x_train_dict = get_subvector_data(ner_representation_limit, embed_name, temp_train_ner)
    x_dev_dict = get_subvector_data(ner_representation_limit, embed_name, tem_dev_ner)
    x_test_dict = get_subvector_data(ner_representation_limit, embed_name, temp_test_ner)

    x_train_dict_sorted = collections.OrderedDict(sorted(x_train_dict.items()))
    x_dev_dict_sorted = collections.OrderedDict(sorted(x_dev_dict.items()))
    x_test_dict_sorted = collections.OrderedDict(sorted(x_test_dict.items()))

    # convert to np array
    x_train_ner = np.array(list(x_train_dict_sorted.values()))
    x_dev_ner = np.asarray(list(x_dev_dict_sorted.values()))
    x_test_ner = np.asarray(list(x_test_dict_sorted.values()))

    del x_train_dict, x_dev_dict, x_test_dict
    del x_train_dict_sorted, x_dev_dict_sorted, x_test_dict_sorted
    gc.collect()

    for iteration in range(1,maxiter):
        print ("Iteration number: ", iteration)

        #set a different seed for each iteration
        set_seed(SEED + iteration)

        for each_problem in target_problems:
            print ("Problem type: ", each_problem)
            print ("__________________")


            early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)

            # reduce learning rate as val loss stops changing
            reduce_lr = ReduceLROnPlateau(monitor=monitor_criteria, factor=0.2,
                              patience=1, min_lr=0.00001, min_delta=0.0001, mode='min')

            best_model_name = f"{drive_dir}/results/"+str(ner_representation_limit)+"-tunedcnn1d-"+str(embed_name)+"-"+str(each_problem)+"-"+"best_model.hdf5"

            checkpoint = ModelCheckpoint(best_model_name, monitor=monitor_criteria, verbose=1,
                save_best_only=True, mode='min',save_weights_only=True)

            callbacks = [early_stopping_monitor, checkpoint,reduce_lr]


            model = proposedmodel(ts_hidden_unit,
                               embed_name, ner_representation_limit,filter_sizes)
            model.fit([x_train_ts, x_train_ner], y_train[each_problem], epochs=num_epoch, verbose=1,
                      validation_data=([x_dev_ts, x_dev_ner], y_dev[each_problem]), callbacks=callbacks, batch_size=batch_size)


            probs, predictions = make_predictions(model, [x_test_ts, x_test_ner])
            print_scores(predictions, probs, y_test[each_problem], embed_name, each_problem, iteration, ts_hidden_unit)

            model.load_weights(best_model_name)

            probs, predictions = make_predictions(model, [x_test_ts, x_test_ner])
            save_scores(predictions, probs, y_test[each_problem], embed_name, each_problem, iteration,
                            ts_hidden_unit, type_of_ner, current_model_name)
            del model
            clear_session()
            gc.collect()


Embedding:  word2vec
Iteration number:  1
Problem type:  mort_hosp
__________________


Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - ETA: 0s - loss: 0.3270 - acc: 0.9025
Epoch 1: val_loss improved from inf to 0.24444, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-tunedcnn1d-word2vec-mort_hosp-best_model.hdf5
15567/15567 [==============================] - 12s 764us/sample - loss: 0.3270 - acc: 0.9025 - val_loss: 0.2444 - val_acc: 0.9156 - lr: 0.0010
Epoch 2/50
15488/15567 [============================>.] - ETA: 0s - loss: 0.2435 - acc: 0.9151
Epoch 2: val_loss improved from 0.24444 to 0.22796, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-tunedcnn1d-word2vec-mort_hosp-best_model.hdf5
15567/15567 [==============================] - 5s 332us/sample - loss: 0.2440 - acc: 0.9148 - val_loss: 0.2280 - val_acc: 0.9134 - lr: 0.0010
Epoch 3/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.2246 - acc: 0.9189
Epoch

Problem type:  mort_icu
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15488/15567 [============================>.] - ETA: 0s - loss: 0.2547 - acc: 0.9352
Epoch 1: val_loss improved from inf to 0.17704, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-tunedcnn1d-word2vec-mort_icu-best_model.hdf5
15567/15567 [==============================] - 11s 721us/sample - loss: 0.2547 - acc: 0.9352 - val_loss: 0.1770 - val_acc: 0.9418 - lr: 0.0010
Epoch 2/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.1763 - acc: 0.9421
Epoch 2: val_loss improved from 0.17704 to 0.17316, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-tunedcnn1d-word2vec-mort_icu-best_model.hdf5
15567/15567 [==============================] - 5s 345us/sample - loss: 0.1764 - acc: 0.9421 - val_loss: 0.1732 - val_acc: 0.9440 - lr: 0.0010
Epoch 3/50
15567/15567 [==============================] - E

Problem type:  los_3
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - ETA: 0s - loss: 0.7333 - acc: 0.6304
Epoch 1: val_loss improved from inf to 0.63415, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-tunedcnn1d-word2vec-los_3-best_model.hdf5
15567/15567 [==============================] - 10s 671us/sample - loss: 0.7333 - acc: 0.6304 - val_loss: 0.6341 - val_acc: 0.6778 - lr: 0.0010
Epoch 2/50
15488/15567 [============================>.] - ETA: 0s - loss: 0.6366 - acc: 0.6577
Epoch 2: val_loss improved from 0.63415 to 0.61310, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-tunedcnn1d-word2vec-los_3-best_model.hdf5
15567/15567 [==============================] - 6s 370us/sample - loss: 0.6367 - acc: 0.6579 - val_loss: 0.6131 - val_acc: 0.6805 - lr: 0.0010
Epoch 3/50
15552/15567 [============================>.] - ETA: 0s - 

Problem type:  los_7
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15488/15567 [============================>.] - ETA: 0s - loss: 0.3579 - acc: 0.9188
Epoch 1: val_loss improved from inf to 0.27518, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-tunedcnn1d-word2vec-los_7-best_model.hdf5
15567/15567 [==============================] - 11s 732us/sample - loss: 0.3573 - acc: 0.9190 - val_loss: 0.2752 - val_acc: 0.9237 - lr: 0.0010
Epoch 2/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.2604 - acc: 0.9222
Epoch 2: val_loss improved from 0.27518 to 0.26422, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-tunedcnn1d-word2vec-los_7-best_model.hdf5
15567/15567 [==============================] - 5s 348us/sample - loss: 0.2604 - acc: 0.9222 - val_loss: 0.2642 - val_acc: 0.9237 - lr: 0.0010
Epoch 3/50
15488/15567 [============================>.] - ETA: 0s - 

...

Problem type:  los_7
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - ETA: 0s - loss: 0.3747 - acc: 0.9169
Epoch 1: val_loss improved from inf to 0.28161, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-tunedcnn1d-concat-los_7-best_model.hdf5
15567/15567 [==============================] - 10s 650us/sample - loss: 0.3747 - acc: 0.9169 - val_loss: 0.2816 - val_acc: 0.9237 - lr: 0.0010
Epoch 2/50
15567/15567 [==============================] - ETA: 0s - loss: 0.2656 - acc: 0.9221
Epoch 2: val_loss improved from 0.28161 to 0.26200, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-tunedcnn1d-concat-los_7-best_model.hdf5
15567/15567 [==============================] - 6s 393us/sample - loss: 0.2656 - acc: 0.9221 - val_loss: 0.2620 - val_acc: 0.9237 - lr: 0.0010
Epoch 3/50
15567/15567 [==============================] - ETA: 0s - loss

Iteration number:  2
Problem type:  mort_hosp
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - ETA: 0s - loss: 0.3507 - acc: 0.9008
Epoch 1: val_loss improved from inf to 0.25197, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-tunedcnn1d-concat-mort_hosp-best_model.hdf5
15567/15567 [==============================] - 10s 637us/sample - loss: 0.3507 - acc: 0.9008 - val_loss: 0.2520 - val_acc: 0.9120 - lr: 0.0010
Epoch 2/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.2476 - acc: 0.9127
Epoch 2: val_loss improved from 0.25197 to 0.23884, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-tunedcnn1d-concat-mort_hosp-best_model.hdf5
15567/15567 [==============================] - 6s 412us/sample - loss: 0.2476 - acc: 0.9127 - val_loss: 0.2388 - val_acc: 0.9120 - lr: 0.0010
Epoch 3/50
15567/15567 [===============

Problem type:  mort_icu
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.2981 - acc: 0.9325
Epoch 1: val_loss improved from inf to 0.21283, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-tunedcnn1d-concat-mort_icu-best_model.hdf5
15567/15567 [==============================] - 10s 635us/sample - loss: 0.2981 - acc: 0.9324 - val_loss: 0.2128 - val_acc: 0.9332 - lr: 0.0010
Epoch 2/50
15488/15567 [============================>.] - ETA: 0s - loss: 0.1896 - acc: 0.9419
Epoch 2: val_loss improved from 0.21283 to 0.17769, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-tunedcnn1d-concat-mort_icu-best_model.hdf5
15567/15567 [==============================] - 6s 386us/sample - loss: 0.1899 - acc: 0.9417 - val_loss: 0.1777 - val_acc: 0.9431 - lr: 0.0010
Epoch 3/50
15552/15567 [============================>.] - ETA: 

Problem type:  los_3
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15488/15567 [============================>.] - ETA: 0s - loss: 0.7381 - acc: 0.6300
Epoch 1: val_loss improved from inf to 0.63954, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-tunedcnn1d-concat-los_3-best_model.hdf5
15567/15567 [==============================] - 9s 574us/sample - loss: 0.7377 - acc: 0.6299 - val_loss: 0.6395 - val_acc: 0.6638 - lr: 0.0010
Epoch 2/50
15552/15567 [============================>.] - ETA: 0s - loss: 0.6380 - acc: 0.6570
Epoch 2: val_loss improved from 0.63954 to 0.61849, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-tunedcnn1d-concat-los_3-best_model.hdf5
15567/15567 [==============================] - 7s 463us/sample - loss: 0.6380 - acc: 0.6569 - val_loss: 0.6185 - val_acc: 0.6782 - lr: 0.0010
Epoch 3/50
15567/15567 [==============================] - ETA: 0s - loss:

Problem type:  los_7
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - ETA: 0s - loss: 0.3660 - acc: 0.9178
Epoch 1: val_loss improved from inf to 0.28723, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-tunedcnn1d-concat-los_7-best_model.hdf5
15567/15567 [==============================] - 10s 672us/sample - loss: 0.3660 - acc: 0.9178 - val_loss: 0.2872 - val_acc: 0.9228 - lr: 0.0010
Epoch 2/50
15488/15567 [============================>.] - ETA: 0s - loss: 0.2677 - acc: 0.9223
Epoch 2: val_loss improved from 0.28723 to 0.26645, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-tunedcnn1d-concat-los_7-best_model.hdf5
15567/15567 [==============================] - 6s 401us/sample - loss: 0.2679 - acc: 0.9222 - val_loss: 0.2664 - val_acc: 0.9237 - lr: 0.0010
Epoch 3/50
15567/15567 [==============================] - ETA: 0s - loss

# Trasformer experiment (INCOMPLETE)

Code is commented out as it crashed colab, it's very slow.

In [ ]:
# def transformer_model(number_of_unit, embedding_name, ner_limit, num_heads):
#     """
#     Arguments:
#         number_of_unit: GRU number of hidden units
#         embedding_name: name of embedding
#         ner_limit: size of patient matrix
#         num_filter: list of 3 CNN filter sizes

#     Outputs:
#         model: multimodal keras model using transformer layer instead of convolution
#     """
#     if embedding_name == "concat":
#         input_dimension = 200
#     else:
#         input_dimension = 100

#     # 24 hours and 104 time-series variable
#     ts_input = Input(shape=(24, 104), name="timeseries_input")
#     input_embeddings = Input(shape=(ner_limit, input_dimension), name="input_embeddings")

#     # Transformer layer for medical entities
#     text_transformer = MultiHeadAttention(num_heads=num_heads, key_dim=input_dimension)(query=input_embeddings, key=input_embeddings, value=input_embeddings)
#     text_transformer = LayerNormalization()(text_transformer)

#     # max pooling layer
#     text_embeddings = GlobalMaxPooling1D()(text_transformer)

#     # GRU layer for time series data
#     x = GRU(number_of_unit)(ts_input)

#     # Concatenate transformer output and GRU output
#     concatenated = Concatenate(axis=1)([x, text_embeddings])

#     # Dense layers
#     concatenated = Dense(512, activation='relu')(concatenated)
#     concatenated = Dropout(0.2)(concatenated)

#     # Fully connected layer
#     preds = Dense(1, activation='sigmoid', use_bias=False, kernel_initializer=GlorotUniform(),
#                   kernel_regularizer=L2(l2=0.01))(concatenated)


#     model = Model(inputs=[ts_input, input_embeddings], outputs=preds)
#     model.compile(loss='binary_crossentropy',
#                   optimizer=Adam(learning_rate=0.001),
#                   metrics=['acc'])

#     return model

In [ ]:
# %%time
# # crashed
# # This run was on CPU

# current_model_name = "Transformer Multimodal"

# for embed_dict, embed_name in zip(embedding_dict, embedding_types):
#     print ("Embedding: ", embed_name)
#     print("=============================")

#     # split emdeddings into training, validation and testing
#     temp_train_ner = dict((k, embed_dict[k]) for k in train_ids)
#     tem_dev_ner = dict((k, embed_dict[k]) for k in dev_ids)
#     temp_test_ner = dict((k, embed_dict[k]) for k in test_ids)

#     # combine medical entiy embeddings vertically, with padding, to create a patient matrix represention
#     x_train_dict = get_subvector_data(ner_representation_limit, embed_name, temp_train_ner)
#     x_dev_dict = get_subvector_data(ner_representation_limit, embed_name, tem_dev_ner)
#     x_test_dict = get_subvector_data(ner_representation_limit, embed_name, temp_test_ner)

#     x_train_dict_sorted = collections.OrderedDict(sorted(x_train_dict.items()))
#     x_dev_dict_sorted = collections.OrderedDict(sorted(x_dev_dict.items()))
#     x_test_dict_sorted = collections.OrderedDict(sorted(x_test_dict.items()))

#     # convert to np array
#     x_train_ner = np.array(list(x_train_dict_sorted.values()))
#     x_dev_ner = np.asarray(list(x_dev_dict_sorted.values()))
#     x_test_ner = np.asarray(list(x_test_dict_sorted.values()))

#     del x_train_dict, x_dev_dict, x_test_dict
#     del x_train_dict_sorted, x_dev_dict_sorted, x_test_dict_sorted
#     gc.collect()

#     for iteration in range(1,maxiter):
#         print ("Iteration number: ", iteration)

#         #set a different seed for each iteration
#         set_seed(SEED + iteration)

#         for each_problem in target_problems:
#             print ("Problem type: ", each_problem)
#             print ("__________________")


#             early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)

#             best_model_name = f"{drive_dir}/results/"+str(ner_representation_limit)+"-transformer-"+str(embed_name)+"-"+str(each_problem)+"-"+"model.hdf5"

#             checkpoint = ModelCheckpoint(best_model_name, monitor=monitor_criteria, verbose=1,
#                 save_best_only=True, mode='min',save_weights_only=True)

#             callbacks = [early_stopping_monitor, checkpoint]


#             model = transformer_model(ts_hidden_unit,
#                                embed_name, ner_representation_limit,8)
#             model.fit([x_train_ts, x_train_ner], y_train[each_problem], epochs=num_epoch, verbose=1,
#                       validation_data=([x_dev_ts, x_dev_ner], y_dev[each_problem]), callbacks=callbacks, batch_size=batch_size)


#             probs, predictions = make_predictions(model, [x_test_ts, x_test_ner])
#             print_scores(predictions, probs, y_test[each_problem], embed_name, each_problem, iteration, ts_hidden_unit)

#             model.load_weights(best_model_name)

#             probs, predictions = make_predictions(model, [x_test_ts, x_test_ner])
#             save_scores(predictions, probs, y_test[each_problem], embed_name, each_problem, iteration,
#                             ts_hidden_unit, type_of_ner, current_model_name)
#             del model
#             clear_session()
#             gc.collect()



Embedding:  word2vec
Iteration number:  1
Problem type:  mort_hosp
__________________
Train on 15567 samples, validate on 2216 samples
Epoch 1/50
15567/15567 [==============================] - ETA: 0s - loss: 0.2740 - acc: 0.9043
Epoch 1: val_loss improved from inf to 0.23750, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-transformer-word2vec-mort_hosp-model.hdf5
15567/15567 [==============================] - 109s 7ms/sample - loss: 0.2740 - acc: 0.9043 - val_loss: 0.2375 - val_acc: 0.9129
Epoch 2/50
15567/15567 [==============================] - ETA: 0s - loss: 0.2394 - acc: 0.9135
Epoch 2: val_loss improved from 0.23750 to 0.22675, saving model to /content/gdrive/My Drive/CS 598/Project/ConvolutionMedicalNer-KB/results/64-transformer-word2vec-mort_hosp-model.hdf5
15567/15567 [==============================] - 112s 7ms/sample - loss: 0.2394 - acc: 0.9135 - val_loss: 0.2267 - val_acc: 0.9183
Epoch 3/50
15567/15567 [=========================